# TRPL t_x lifetime H5 

# Parameters

In [ ]:
settings = dict(
    filename = "data/180611_143555_Picoharp_MCL_2DSlowScan.h5",
    kk_start = 0,
    kk_stop = 100,
    bg_sub = True,
    e_exp = 1.0,
    spatial_blur = False,
    blur_sigma = 1.0,
)

# Load Data

In [ ]:
import numpy as np

In [ ]:
class AttrDict(dict):
    def __init__(self, *args, **kwargs):
        super(AttrDict, self).__init__(*args, **kwargs)
        self.__dict__ = self

In [ ]:
import h5py
h5py.enable_ipython_completer()
D = AttrDict()
D.settings = settings
D.dat = h5py.File(D.settings['filename'])
D.M = D.dat['measurement/Picoharp_MCL_2DSlowScan']

D.cr0 = D.dat['hardware/picoharp/settings'].attrs['count_rate0']
D.rep_period_s = 1.0/D.cr0
D.time_bin_resolution = D.dat['hardware/picoharp/settings'].attrs['Resolution']*1e-12
D.num_hist_chans = int(np.ceil(D.rep_period_s/D.time_bin_resolution))

# truncate data to only show the time period associated with rep-rate of laser
def norm_2d(X):
    return X / np.reshape(np.max(X, 2), X.shape[:-1] + (1,))

D.time_trace_map = norm_2d(np.array(D.M['time_trace_map'][0,:,:,:]))
D.integrated_count_map = D.time_trace_map.sum(axis=2)
D.time_array = np.array(D.M['time_array'])

D.hyperspec_data = D.time_trace_map[:,:,0:D.num_hist_chans]
#display_image = integrated_count_map
D.spec_x_array = D.time_array[0:D.num_hist_chans]


D.imshow_extent = np.array(D.M['imshow_extent'])
D.h_array = np.array(D.M['h_array'])
D.v_array = np.array(D.M['v_array'])

# Compute Lifetime Map 

In [ ]:
def t_x_calc(time_array, time_trace_map, kk_start, kk_stop, x=1-0.36787944117, bgsub=True):

    kk_bg_max = int(4*kk_start/5)

    bg_slice = slice(0,kk_bg_max) #fit_config.bg_slice #slice(0,kk_bg_max/2)

    if len(time_trace_map.shape) == 4: #if 4d (3d + time) data
        Nz, Ny, Nx, Nt = time_trace_map.shape 
        bg = np.average(time_trace_map[:,:,:,bg_slice], axis=3).reshape(Nz,Ny,Nx,1)
        T = np.array(time_trace_map[:,:,:,kk_start:kk_stop], dtype=float) # copy array
        if bgsub:
            T -= bg

        t_x_map = time_array[  np.argmin(
                               np.abs( np.cumsum(T, axis=3)/ 
                                          np.sum(T, axis=3).reshape(Nz, Ny, Nx,1)
                                          - x), axis=3)]
    else: #if 3d (2d + time) data
        Ny, Nx, Nt = time_trace_map.shape
        bg = np.mean(time_trace_map[:,:,bg_slice], axis=2).reshape(Ny, Nx,1)
        
        T = np.array(time_trace_map[:,:,kk_start:kk_stop], dtype=float) # copy array
        if bgsub:
            T -= bg

        t_x_map =  time_array[np.argmin(
                                np.abs(np.cumsum(T, axis=2)/
                                                     np.sum(T, axis=2).reshape(Ny, Nx,1) 
                                                         - x ), axis=2)]
        
    return t_x_map, bg

In [ ]:
from scipy.ndimage.filters import gaussian_filter

if D.settings['spatial_blur']:
    s = D.settings['blur_sigma']
    D.data_map = gaussian_filter(D.time_trace_map, sigma=(s,s,0))
else:
    D.data_map = D.time_trace_map


D.tau_x_map, D.bg = t_x_calc(D.time_array, 
                                   D.data_map,
                                   x = 1 - np.exp(-1*D.settings['e_exp']),
                                   kk_start=D.settings['kk_start'], 
                                   kk_stop=D.settings['kk_stop'],
                                   bgsub=D.settings['bg_sub'])

D.time_trace_map_bgsub = D.data_map - D.bg


# Intensity and Lifetime plot 

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

from mpl_toolkits.axes_grid1 import make_axes_locatable

def colorbar(mappable, loc='right', **kwargs):
    ax = mappable.axes
    fig = ax.figure
    divider = make_axes_locatable(ax)
    cax = divider.append_axes(loc, size="5%", pad=0.05)
    return fig.colorbar(mappable, cax=cax, **kwargs)

In [ ]:
plt.figure()

S = D.M['settings']

ax = plt.subplot(121)
im = plt.imshow(D.time_trace_map_bgsub.sum(axis=2), origin='lower', 
           extent=D.imshow_extent, cmap=plt.cm.YlOrBr_r)
colorbar(im, label='Total Counts')
#plt.colorbar(label='Total Counts')
ax.set_xlabel(S.attrs['h_axis'] +" (um)")
ax.set_ylabel(S.attrs['v_axis'] +" (um)")

ax = plt.subplot(122)
im = plt.imshow(D.tau_x_map, origin='lower', extent=D.imshow_extent)
#plt.colorbar(label='Lifetime (ns)')
colorbar(im, label='Lifetime (ns)')
ax.set_xlabel(S.attrs['h_axis'] +" (um)")
ax.set_ylabel(S.attrs['v_axis'] +" (um)")


plt.tight_layout()

# Time trace plot

In [ ]:
plt.figure()

img_ax = plt.subplot(221)
im = img_ax.imshow(D.time_trace_map_bgsub.sum(axis=2), origin='lower', 
           extent=D.imshow_extent, cmap=plt.cm.YlOrBr_r)
colorbar(im, label='Total Counts')
img_ax.set_xlabel(S.attrs['h_axis'] +" (um)")
img_ax.set_ylabel(S.attrs['v_axis'] +" (um)")

tau_img_ax = plt.subplot(222)
im = tau_img_ax.imshow(D.tau_x_map, origin='lower', extent=D.imshow_extent)
#plt.colorbar(label='Lifetime (ns)')
colorbar(im, label='Lifetime (ns)')
tau_img_ax.set_xlabel(S.attrs['h_axis'] +" (um)")
tau_img_ax.set_ylabel(S.attrs['v_axis'] +" (um)")


time_ax = plt.subplot(212)
time_ax.set_xlabel('Time (ns)')
time_ax.set_ylabel('Counts')

for n,(j,i) in enumerate(D.settings['test_points']):
    x = D.h_array[i]
    y = D.v_array[j]
    for ax in img_ax, tau_img_ax:
        ax.plot( x, y, 'x', color="C{}".format(n))

    time_ax.semilogy(D.time_array, D.time_trace_map_bgsub[j,i,:])
    
plt.tight_layout()

In [ ]:
#roi

In [ ]:
#full

# combined (zoomed)